In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
import cv2
import numpy as np

In [3]:
from matplotlib import pyplot as plt

In [4]:
import os

In [5]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

## Testing basic H threshold

In [6]:
# video_path = "./ignore/lab_green_screen/suyash_oneplus.mp4"
# video_path = "./video/test.mp4"
video_path = "./ignore/lab_green_screen/ankesh_oneplus.mp4"
# video_path = "./ignore/Green Screen/JA0A7025.MOV"
cap = cv2.VideoCapture(video_path)

In [7]:
ret,IMG = cap.read()
IMG = cv2.resize(IMG,(IMG.shape[1]//2,IMG.shape[0]//2))

In [8]:
show_images([("a",IMG)])

In [ ]:
# def segment_hls(orig,params,al,bet,gm,mult):
#     h_mean,h_std,l_mean,l_std,s_mean,s_std = params
#     hls_im = cv2.cvtColor(orig,cv2.COLOR_BGR2HLS)
#     a = (hls_im[:,:,0]>(h_mean-mult*h_std))
#     b = (hls_im[:,:,0]<(h_mean+mult*h_std))
#     h_mask = a*b
#     a = (hls_im[:,:,1]>(l_mean-mult*l_std))
#     b = (hls_im[:,:,1]<(l_mean+mult*l_std))
#     l_mask = a*b
#     a = (hls_im[:,:,2]>(s_mean-mult*s_std))
#     b = (hls_im[:,:,2]<(s_mean+mult*s_std))
#     s_mask = a*b
#     hls_mask = (al*h_mask + bet*l_mask + gm*s_mask)
# #     return hls_mask
#     hls_mask = hls_mask.clip(0.0,1.0)
#     hls_mask = 1 - hls_mask
#     return hls_mask

In [9]:
def segment_ycrcb(orig,params,tola,tolb):
    ycrcb_im = cv2.cvtColor(orig,cv2.COLOR_BGR2YCrCb)
    Cb_key,Cr_key = params
    blue = ycrcb_im[:,:,2]
    red = ycrcb_im[:,:,1]

    diffbsq = (blue-Cb_key)**2
    diffrsq = (red-Cr_key)**2
    dist = np.sqrt(diffbsq+diffrsq).astype(np.float32)

    mask = ((dist-tola)/(tolb-tola)).astype(np.float32)
    mask[dist<tola]=0.0
    mask[dist>tolb]=1.0
    return mask

In [10]:
def get_region(img):
    r = cv2.selectROI("Select window of background",img=img,fromCenter=False,showCrossair=False)
    cv2.destroyAllWindows()
    return r

In [11]:
def get_params_hls(img,region):
    hls_img = cv2.cvtColor(img,cv2.COLOR_BGR2HLS).astype(np.float32)
    r = [int(x) for x in region]
    region = hls_img[int(region[1]):int(region[1]+region[3]), int(region[0]):int(region[0]+region[2])]
    h_mean,l_mean,s_mean = np.mean(region,axis=(0,1))
    h_std,l_std,s_std = np.std(region,axis=(0,1))
    return [h_mean,h_std,l_mean,l_std,s_mean,s_std]

In [12]:
def get_params_ycrcb(img,region):
    ycrcb_img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb).astype(np.float32)
    cv2.destroyAllWindows()
    r = [int(x) for x in region]
    region = ycrcb_img[int(region[1]):int(region[1]+region[3]), int(region[0]):int(region[0]+region[2])]
    y_mean,Cr_mean,Cb_mean = np.mean(region,axis=(0,1))
    y_std,Cr_std,Cb_std = np.std(region,axis=(0,1))
    return [Cb_mean,Cr_mean]

In [13]:
def brighten(img,alpha,beta,gamma):
    cor_img = img.astype(np.float32)
    bright_img = alpha*cor_img + beta
    gam_cor = np.power(bright_img/255,gamma)*255
    bright_img = gam_cor.clip(0,255).astype(np.uint8)
    return bright_img

In [14]:
def mod_mask(mask,low,high):
    mask = mask.copy()
    mask[mask>high]=1.0
    mask[mask<low] = 0.0
    return mask

In [34]:
def process(img,bg,param_ycrcb,param_hls,tola,tolb,low_thresh,high_thresh,alpha,beta,gamma,sz,space,erode_sz):
    ##ENSURE THAT param_ycrcb and param_hls correspond to brigthened img
    
#     brimg = brighten(img,alpha,beta,gamma)
    brimg = img
    brimg = cv2.bilateralFilter(brimg,sz,space,space)
    
    mask = segment_ycrcb(brimg,param_ycrcb,tola,tolb)
    mask = mod_mask(mask,low_thresh,high_thresh)
    
#     kernel = np.ones((erode_sz,erode_sz),np.uint8)
#     mask = cv2.erode(mask,kernel,iterations = 1)
    
    mask = np.expand_dims(mask,-1)
    
    new_img = (mask*img+(1-mask)*bg).astype(np.uint8)
    return (mask,new_img)
#     return new_img
#     cv2.imshow("masked",new_img)
#     cv2.imshow("mask",mask)
#     ret = cv2.waitKey(100)

In [16]:
def write_alpha_img(img,mask,path):
    r_channel, g_channel, b_channel = cv2.split(img)
    alpha = (mask*255).astype(np.uint8)
    img_RGBA = cv2.merge((r_channel, g_channel, b_channel, alpha))
    cv2.imwrite(path, img_RGBA)
    return img_RGBA

In [17]:
WHITE = np.ones(IMG.shape,dtype=np.int8)*255
region = get_region(IMG)

In [28]:
param_hls = get_params_hls(IMG,region)

In [25]:
param_ycrcb = get_params_ycrcb(IMG,region)

In [ ]:
show_images([("x",IMG)])

In [ ]:
im = cv2.bilateralFilter(IMG,5,100,100)
# im = cv2.bilateralFilter(IMG,1,1,1)
show_images([("orig",IMG),("x",im)])

In [33]:
%timeit process(IMG,WHITE,param_ycrcb,param_hls,tola,tolb,low_thresh,high_thresh,alpha,beta,gamma,bl_sz,bl_space,erode_sz)

10 loops, best of 3: 49.9 ms per loop


In [23]:
def test(img,bg,region,tola,tolb,low_thresh,high_thresh,alpha,beta,gamma,sz,space,erode_sz):
    
    brimg = brighten(img,alpha,beta,gamma)
    brimg = cv2.bilateralFilter(brimg,sz,space,space)
    
    param_hls = get_params_hls(brimg,region)
    param_ycrcb = get_params_ycrcb(brimg,region)
    
    mask = segment_ycrcb(brimg,param_ycrcb,tola,tolb)
    mask = mod_mask(mask,low_thresh,high_thresh)
    
    kernel = np.ones((erode_sz,erode_sz),np.uint8)
    mask = cv2.erode(mask,kernel,iterations = 1)
    
    mask = np.expand_dims(mask,-1)
    new_img = (mask*img+(1-mask)*bg).astype(np.uint8)
    
#     cv2.imshow("orig",img)
#     cv2.imshow("filterimg",brimg)
#     cv2.imshow("Mask",mask)
#     cv2.imshow("new",new_img)
#     cv2.waitKey(1000)


In [ ]:
cv2.destroyAllWindows()

In [26]:
tola=16
tolb=50
low_thresh = 0.05
high_thresh = 0.25
alpha = 1.0
beta = 0
gamma = 1.0
bl_sz=5
bl_space=200
erode_sz=0

In [21]:
%timeit test(img=(IMG),bg=(WHITE),region=(region),tola=16,tolb=50,low_thresh=0.05,high_thresh=0.25,alpha=1.0,beta=0.0,gamma=1.0,sz=5,space=200,erode_sz=0)

10 loops, best of 3: 62.3 ms per loop


In [ ]:
interact(test,img=fixed(IMG),bg=fixed(WHITE),region=fixed(region),tola=16,tolb=50,low_thresh=0.05,high_thresh=0.25,alpha=1.0,beta=0.0,gamma=1.0,sz=5,space=200,erode_sz=0)

In [ ]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
outpath = "./ankesh_vid/"
outp_img = outpath+"image/"
outp_vid = outpath+"video.avi"
os.makedirs(outp_img,exist_ok=True)
os.makedirs(outpath,exist_ok=True)

output = []
i = 0

tola=16
tolb=50
low_thresh = 0.05
high_thresh = 0.25
alpha = 1.0
beta = 0
gamma = 1.0
bl_sz=5
bl_space=200
erode_sz=0

cap = cv2.VideoCapture(video_path)
fps = cap.get(5)
ret,img = cap.read()
img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
region = get_region(img)

vid = cv2.VideoWriter(outp_vid,fourcc, fps, (img.shape[1],img.shape[0]))
WHITE = np.ones(img.shape,dtype=np.int8)*255

brimg = brighten(img,alpha,beta,gamma)
brimg = cv2.bilateralFilter(brimg,bl_sz,bl_space,bl_space)

param_hls = get_params_hls(brimg,region)
param_ycrcb = get_params_ycrcb(brimg,region)

output = []

while(True):
    ret,img = cap.read()
    if not ret:
        break
    img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    
    mask,new_img = process(img,WHITE,param_ycrcb,param_hls,tola,tolb,
                           low_thresh,high_thresh,alpha,beta,gamma,bl_sz,bl_space,erode_sz)
    
    
    
    cv2.imshow("masked",new_img)
    cv2.imshow("mask",mask)
    
    img_name = outp_img+"img_%04d.png"%i
    rgba_img = write_alpha_img(img,mask,img_name)

    vid.write(new_img)
    output.append(new_img)
    
    ret = cv2.waitKey(1)
    if ret==27:
        break
    i+=1
cv2.destroyAllWindows()
vid.release()